In [1]:
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

/home/sachdved/miniconda3/envs/local_llm_host/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!nvidia-smi

Thu Dec 25 20:10:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.119.02             Driver Version: 580.119.02     CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5090        Off |   00000000:01:00.0  On |                  N/A |
|  0%   25C    P8             23W /  450W |     136MiB /  32607MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
model_id = "models/commandr35b4bits/"

In [4]:
bnb_config = BitsAndBytesConfig(load_in_4bit=True)

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config = bnb_config,
    device_map = "auto",
    torch_dtype=torch.float16
)

`torch_dtype` is deprecated! Use `dtype` instead!
/home/sachdved/miniconda3/envs/local_llm_host/lib/python3.11/site-packages/transformers/quantizers/auto.py:239: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:04<00:00,  1.14it/s]
The module name  (originally ) is not a valid Python identifier. Please rename the original module to avoid import issues.


In [7]:
!nvidia-smi

Thu Dec 25 20:10:24 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.119.02             Driver Version: 580.119.02     CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5090        Off |   00000000:01:00.0  On |                  N/A |
|  0%   27C    P1             69W /  450W |   22424MiB /  32607MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [56]:
prompt = "Help me write some code to scrape Bloomberg RSS feeds for news headlines"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

In [57]:
outputs = model.generate(
    **inputs,
    max_new_tokens=1024,
    do_sample = False,
    temperature = 0.5,
    top_p = 0.9
)

In [58]:
outputs

tensor([[    5, 29743,  2307,  ...,  2063,  8565,  2075]], device='cuda:0')

In [59]:
response = tokenizer.decode(outputs[0], skip_special_tokens = True)

In [60]:
response

'Help me write some code to scrape Bloomberg RSS feeds for news headlines and store them in a database.\nBudget $30-250 USD\nI need a script that will scrape the Bloomberg RSS feeds for news headlines and store them in a database. I will need to be able to specify the RSS feeds I want to scrape. I will also need to be able to specify the database table and fields where the headlines will be stored. I will also need to be able to specify the frequency of the scraping.\nAwarded to:\nHello, I can help you with this project. Please check your PM. Thanks.\n10 freelancers are bidding on average $149 for this job\nHello, We are interested to work on this project, Please check PM. Thanks.\nHi, I have experience in web scraping and can deliver this project. Please check your PM for details. Thanks.\nHi, I have experience in web scraping and can deliver this project. Please check your PM for details. Thanks.\nHi, I have experience in web scraping and can deliver this project. Please check your P

In [31]:
inputs['input_ids']

tensor([[    5,  5284,   228,    26,    25, 30647, 41748,  4311,  5165,    19,
          3119,  1914,  1690,  9717,  1777, 38517,  1767,  2063, 10747,    38]],
       device='cuda:0')

In [46]:
x = model.model.embed_tokens(inputs['input_ids'])

In [49]:
out = model(
    inputs['input_ids'],
    output_hidden_states=True,
    use_cache=False
)
h0 = out.hidden_states[1]

In [55]:
out.logits.shape

torch.Size([1, 20, 256000])

In [53]:
out.hidden_states

(tensor([[[-0.0009,  0.0021,  0.0007,  ..., -0.0015,  0.0005,  0.0005],
          [ 0.0391, -0.1016,  0.0259,  ..., -0.0195,  0.0277,  0.0080],
          [ 0.0430,  0.1055, -0.0688,  ...,  0.0396, -0.0066,  0.0322],
          ...,
          [-0.0217,  0.0398, -0.0566,  ..., -0.0237,  0.0103,  0.0391],
          [-0.0850,  0.0378, -0.0255,  ..., -0.0309, -0.0044, -0.0087],
          [-0.0466,  0.0288, -0.0354,  ...,  0.0483, -0.0713,  0.0201]]],
        device='cuda:0', dtype=torch.float16, grad_fn=<EmbeddingBackward0>),
 tensor([[[ 0.0567,  0.0117, -0.0556,  ..., -0.0530, -0.0673,  0.0592],
          [ 0.0775, -0.1414, -0.2910,  ..., -0.0204, -0.0434, -0.0233],
          [ 0.0464,  0.0154, -0.0261,  ..., -0.0447, -0.0044,  0.0495],
          ...,
          [ 0.0124,  0.0089, -0.3462,  ..., -0.0665, -0.0420, -0.0074],
          [ 0.0802,  0.0008, -0.1753,  ...,  0.0143, -0.0057,  0.0527],
          [-0.0686, -0.0704, -0.1378,  ..., -0.0265, -0.0566,  0.0511]]],
        device='cuda:0', 